In [2]:
import os
import pathlib
import sys
import numpy as np
from os import path
import glob, matplotlib.pyplot as plt, matplotlib.patches as patches
import xml.etree.ElementTree as et

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dir = "./train"
val_dir = "./val"


train_data = image_dataset_from_directory(train_dir,label_mode = "binary",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = True,seed = 42)
val_data = image_dataset_from_directory(val_dir,label_mode = "binary",
                                          image_size = (224,224),batch_size = 32,
                                         shuffle = False,seed = 42)

Found 3300 files belonging to 2 classes.
Found 1700 files belonging to 2 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import layers

tf.get_logger().setLevel('ERROR') # Suppress TensorFlow logging

In [5]:
early_stop = tf.keras.callbacks.EarlyStopping(monitor = "val_loss",patience = 6,
                                             min_delta = 0.0001)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss",factor = 0.2,
                                                patience = 4,min_lr = 1e-7)

In [16]:
model = tf.keras.applications.EfficientNetB7(include_top = False)
model.trainable = False
# My GPU Memmory is short ㅠㅠ

<h1> Modeling </h1>

In [17]:
data_aug = tf.keras.Sequential([
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomFlip("horizontal")
],name = "data_augmentation_layer")


In [18]:
inputs = layers.Input(shape = (224,224,3),name = "input_layer")
x = data_aug(inputs)
x = model(x)
x = layers.GlobalAvgPool2D(name = "pooling_layer")(x)

x = layers.Dense(32,activation = "relu",kernel_initializer = tf.keras.initializers.he_normal())(x)
x = layers.Dense(1)(x)
outputs = layers.Activation("sigmoid",dtype = tf.float32)(x)
model = tf.keras.Model(inputs,outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation_layer (Se  (None, 224, 224, 3)      0         
 quential)                                                       
                                                                 
 efficientnetb7 (Functional)  (None, None, None, 2560)  64097687 
                                                                 
 pooling_layer (GlobalAverag  (None, 2560)             0         
 ePooling2D)                                                     
                                                                 
 dense_2 (Dense)             (None, 32)                81952     
                                                                 
 dense_3 (Dense)             (None, 1)                 33  

In [19]:
model.compile(loss = tf.keras.losses.BinaryCrossentropy(),
              optimizer = tf.keras.optimizers.Adam(),
              metrics = ["accuracy"])

In [20]:
from sklearn.utils import class_weight
y = np.concatenate([y for x, y in train_data], axis=0)
weights = class_weight.compute_class_weight(class_weight = 'balanced',
                                            classes  = np.unique(np.ravel(y)),
                                            y = np.ravel(y))

In [21]:
weights[0]

5.5

In [22]:
weights = {0: weights[0], 1:weights[1]}

In [23]:
weights

{0: 5.5, 1: 0.55}

In [24]:
TensorB = tf.keras.callbacks.TensorBoard('./board')

In [25]:
history_1 = model.fit(train_data,epochs =20 , validation_data = val_data, 
                      callbacks = [early_stop,reduce_lr, TensorB], 
                      class_weight=weights)

Epoch 1/20
104/104 [==============================] - 109s 908ms/step - loss: 0.3971 - accuracy: 0.7958 - val_loss: 0.2607 - val_accuracy: 0.9012 - lr: 0.0010
Epoch 2/20
104/104 [==============================] - 82s 790ms/step - loss: 0.2851 - accuracy: 0.8655 - val_loss: 0.2320 - val_accuracy: 0.9112 - lr: 0.0010
Epoch 3/20
104/104 [==============================] - 81s 775ms/step - loss: 0.2784 - accuracy: 0.8733 - val_loss: 0.3046 - val_accuracy: 0.8824 - lr: 0.0010
Epoch 4/20
104/104 [==============================] - 81s 782ms/step - loss: 0.2305 - accuracy: 0.8897 - val_loss: 0.2452 - val_accuracy: 0.9029 - lr: 0.0010
Epoch 5/20
104/104 [==============================] - 77s 743ms/step - loss: 0.2137 - accuracy: 0.8891 - val_loss: 0.1943 - val_accuracy: 0.9241 - lr: 0.0010
Epoch 6/20
104/104 [==============================] - 79s 758ms/step - loss: 0.2109 - accuracy: 0.8955 - val_loss: 0.1924 - val_accuracy: 0.9176 - lr: 0.0010
Epoch 7/20
104/104 [==============================]

In [27]:
!pip install pillow

In [58]:
import PIL.Image as pilimg

im = pilimg.open( 'test.jpg' )
 
# Display image
im = im.resize((224,224))
im.show()
# Fetch image pixel data to numpy array
pix = np.array(im).reshape(1,224,224,3)

print(pix.shape)

(1, 224, 224, 3)


In [60]:
model.predict(pix)

1/1 [==============================] - 4s 4s/step


array([[0.99988997]], dtype=float32)

In [61]:
import PIL.Image as pilimg

im = pilimg.open( 'test2.jpg' )
 
# Display image
im = im.resize((224,224))
im.show()
# Fetch image pixel data to numpy array
pix = np.array(im).reshape(1,224,224,3)

print(pix.shape)

(1, 224, 224, 3)


In [62]:
model.predict(pix)

1/1 [==============================] - 0s 38ms/step


array([[0.01809156]], dtype=float32)

In [63]:
import PIL.Image as pilimg

im = pilimg.open( 'test3.jpg' )
 
# Display image
im = im.resize((224,224))
im.show()
# Fetch image pixel data to numpy array
pix = np.array(im).reshape(1,224,224,3)

print(pix.shape)

(1, 224, 224, 3)


In [64]:
model.predict(pix)

1/1 [==============================] - 0s 59ms/step


array([[0.8446484]], dtype=float32)

In [67]:
model.save('savedmodel/my_model')

TypeError: Unable to serialize [2.0896919 2.1128857 2.1081853] to JSON. Unrecognized type <class 'tensorflow.python.framework.ops.EagerTensor'>.